In [1]:
import numpy as np
import re
import open3d as o3d
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def parse_x_file(filepath):
    with open(filepath, 'r') as f:
        text = f.read()

    # --- Извлечение RV_Calibration (матрица перехода из мировой системы координат в систему координат камеры) ---
    rv_match = re.search(r'RV_Calibration\s*{\s*((?:-?\d+\.\d+[,;]\s*){16})', text)
    if not rv_match:
        raise ValueError("RV_Calibration не найдена")

    matrix_values = list(map(float, re.findall(r'-?\d+\.\d+', rv_match.group(1))))
    rv_matrix = np.array(matrix_values).reshape((4, 4))

    # --- Извлечение количества вершин ---
    mesh_header = re.search(r'Mesh\s+\w+\s*{\s*(\d+);', text)
    if not mesh_header:
        raise ValueError("Mesh-блок не найден")
    num_vertices = int(mesh_header.group(1))
    print(f"num_vertices: {num_vertices}")

    # --- Извлечение самих вершин (Nx3) ---
    vertex_pattern = r'(-?\d+\.\d+);(-?\d+\.\d+);(-?\d+\.\d+);[;,]'
    vertex_data = re.findall(vertex_pattern, text)
    if len(vertex_data) < num_vertices:
        raise ValueError("Недостаточно вершин найдено")
    vertices = np.array(vertex_data[:num_vertices], dtype=np.float32)

    # --- Извлечение количества треугольников ---
    face_start_idx = text.find(vertex_data[num_vertices - 1][2])
    face_block_match = re.search(r'(\d+);\s*(3;[^\}]+?);', text[face_start_idx:], re.DOTALL)
    if not face_block_match:
        raise ValueError("Блок треугольников не найден")
    num_faces = int(face_block_match.group(1))
    print(f"num_faces: {num_faces}")

    # --- Извлечение индексов треугольников ---
    face_pattern = r'3;(\d+),(\d+),(\d+);[;,]'
    face_data = re.findall(face_pattern, text)
    if len(face_data) < num_faces:
        raise ValueError("Недостаточно треугольников найдено")
    faces = np.array(face_data[:num_faces], dtype=np.int32)

    # --- Извлечение UV-координат ---
    uv_match = re.search(r'MeshTextureCoords\s*{\s*(\d+);', text)
    if not uv_match:
        raise ValueError("UV-координаты не найдены")
    num_uvs = int(uv_match.group(1))
    uv_data = re.findall(r'(-?\d+\.\d+);(-?\d+\.\d+);[;,]', text[uv_match.start():])
    if len(uv_data) >= num_uvs:
        raise ValueError("Недостаточно UV-координат")
    uvs = np.array(uv_data[:num_uvs], dtype=np.float32)

    return {
        "vertices": vertices,
        "faces": faces,
        "uvs": uvs,
        "rv_matrix": rv_matrix
    }

In [3]:
def apply_calibration(vertices, matrix):
    vertices_h = np.hstack([vertices, np.ones((vertices.shape[0], 1))])  # [x, y, z, 1]
    transformed = vertices_h @ matrix.T
    return transformed[:, :3]

In [4]:
def create_voxel_grid(points, grid_size=256):
    # Нормализация координат в [0, 1]
    min_bounds = points.min(axis=0)
    max_bounds = points.max(axis=0)
    norm_points = (points - min_bounds) / (max_bounds - min_bounds + 1e-5)

    # Масштабирование в voxel grid
    voxel_coords = np.floor(norm_points * (grid_size - 1)).astype(int)

    # Заполнение воксельной сетки
    voxels = np.zeros((grid_size, grid_size, grid_size), dtype=bool)
    for x, y, z in voxel_coords:
        voxels[x, y, z] = True

    return voxels

In [5]:
def visualize_voxel_grid(voxels):
    coords = np.argwhere(voxels)
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(coords)
    o3d.visualization.draw_geometries([pc])

In [6]:
VOXEL_GRID_SIZE = 256

In [7]:
ROOT_DIR = os.path.dirname(os.path.abspath(__file__))
filepath = os.path.join(ROOT_DIR, 'data\\teapot_1.x')

NameError: name '__file__' is not defined

In [8]:
filepath = os.path.abspath('data\\teapot_1.x')

In [28]:
vertices, matrix = parse_x_file(filepath)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



ValueError: Недостаточно вершин найдено

In [12]:
matrix

array([[ 0.672113, -0.737551,  0.065438,  0.      ],
       [ 0.730691,  0.674961,  0.10256 ,  0.      ],
       [-0.119812, -0.021117,  0.992572,  0.      ],
       [ 0.      ,  0.      ,  0.      ,  1.      ]])

In [ ]:
voxels = create_voxel_grid(vertices, VOXEL_GRID_SIZE)

In [ ]:
visualize_voxel_grid(voxels)